# Chapter 22: The `keyof` Operator

---

## 22.1 Understanding `keyof`

The `keyof` operator is one of TypeScript's most essential type operators. It produces a union type of all known, public property keys of a given type. This operator serves as the bridge between runtime object keys and compile-time type safety, enabling dynamic yet type-safe property access patterns.

### 22.1.1 Basic Usage

The `keyof` operator takes a type operand and returns a union of its keys as string (or symbol) literal types.

```typescript
// Basic keyof usage
interface User {
  id: number;
  name: string;
  email: string;
}

type UserKeys = keyof User;
// "id" | "name" | "email"

// Works with type aliases too
type Point = {
  x: number;
  y: number;
};

type PointKeys = keyof Point;
// "x" | "y"

// Using with variables (typeof + keyof)
const config = {
  apiUrl: "https://api.example.com",
  timeout: 5000,
  retries: 3
};

type ConfigKeys = keyof typeof config;
// "apiUrl" | "timeout" | "retries"
```

**How `keyof` Works:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                         keyof Operator                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Input Type:                    Output Type:                       │
│   ┌─────────────────────┐         ┌─────────────────────┐          │
│   │ interface User {    │         │ type UserKeys =     │          │
│   │   id: number;       │         │   "id" | "name" |   │          │
│   │   name: string;     │    ──►  │   "email";          │          │
│   │   email: string;      │         │                     │          │
│   │ }                   │         │ Union of all keys   │          │
│   └─────────────────────┘         └─────────────────────┘          │
│                                                                     │
│   Key Characteristics:                                              │
│   • Returns union of string literal types                         │
│   • Includes all public, own properties                             │
│   • Works with objects, classes, and arrays                         │
│   • Preserves readonly and optional modifiers (in the value type)   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 22.1.2 `keyof` with Object Types

When applied to object types, `keyof` extracts all property keys including those from index signatures.

```typescript
// Interface with specific properties
interface Product {
  id: number;
  name: string;
  price: number;
}

type ProductKey = keyof Product;
// "id" | "name" | "price"

// With index signatures
interface Dictionary {
  [key: string]: string;
}

type DictKey = keyof Dictionary;
// string (not a specific union, because any string is valid)

// Mixed: specific properties + index signature
interface Config {
  version: string;
  [setting: string]: string | number;
}

type ConfigKey = keyof Config;
// string | "version"
// Note: "version" is included in string, so effectively string

// Numeric index signatures
interface NumberIndexed {
  [index: number]: boolean;
}

type NumberKey = keyof NumberIndexed;
// number

// Array keyof (returns number | "length" | "toString" | ...)
type ArrayKey = keyof string[];
// number | "length" | "toString" | "toLocaleString" | "pop" | "push" | ...

// Tuple keyof
type TupleKey = keyof [string, number];
// "0" | "1" | "length" | "toString" | ...
// Note: "0" and "1" are string literals, not numbers
```

**Practical Example - Property Validation:**

```typescript
// Type-safe property checker
function hasProperty<T extends object, K extends PropertyKey>(
  obj: T,
  key: K
): obj is T & Record<K, unknown> {
  return key in obj;
}

// Using with keyof for compile-time safety
function getProperty<T extends object, K extends keyof T>(
  obj: T,
  key: K
): T[K] {
  return obj[key];
}

const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com"
};

// TypeScript knows the return type based on the key
const id: number = getProperty(user, "id");
const name: string = getProperty(user, "name");

// getProperty(user, "phone"); // ❌ Error: "phone" is not assignable to keyof User
```

### 22.1.3 `keyof` with Interfaces

`keyof` works seamlessly with interfaces, including those with optional properties, readonly modifiers, and inheritance.

```typescript
// Interface with optional and readonly
interface ReadonlyUser {
  readonly id: number;
  name: string;
  email?: string;
}

type ReadonlyUserKeys = keyof ReadonlyUser;
// "id" | "name" | "email"
// Note: keyof doesn't distinguish optional vs required or readonly

// Interface inheritance
interface Entity {
  id: number;
  createdAt: Date;
}

interface User extends Entity {
  name: string;
  email: string;
}

type UserKeys = keyof User;
// "id" | "createdAt" | "name" | "email"
// Includes inherited keys

// Class as type
class Person {
  constructor(
    public name: string,
    private age: number,  // private is excluded from keyof
    protected email: string  // protected is excluded from keyof
  ) {}
}

type PersonKeys = keyof Person;
// "name" only (public members only)
// Note: In recent TypeScript versions, private/protected are included
// but accessing them from outside the class is an error

// Class with methods
class Service {
  name: string = "";
  
  getName(): string {
    return this.name;
  }
  
  setName(name: string): void {
    this.name = name;
  }
}

type ServiceKeys = keyof Service;
// "name" | "getName" | "setName"
// Methods are included!
```

---

## 22.2 `keyof` with Generics

Combining `keyof` with generic types creates powerful, reusable type-safe utilities that can work with any object structure while maintaining type constraints.

### 22.2.1 Generic Constraints with `keyof`

Use `keyof` to constrain generic parameters to valid keys of a type, ensuring type safety in generic functions.

```typescript
// Generic function with keyof constraint
function getValue<T extends object, K extends keyof T>(
  obj: T,
  key: K
): T[K] {
  return obj[key];
}

// Usage with different types
interface User {
  name: string;
  age: number;
}

interface Product {
  title: string;
  price: number;
}

const user: User = { name: "John", age: 30 };
const product: Product = { title: "Widget", price: 99.99 };

// TypeScript infers the correct return type
const userName = getValue(user, "name");     // Type: string
const userAge = getValue(user, "age");       // Type: number
const productTitle = getValue(product, "title"); // Type: string

// getValue(user, "price"); // ❌ Error: "price" is not assignable to keyof User

// Multiple key access
function getValues<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): T[K][] {
  return keys.map(key => obj[key]);
}

const values = getValues(user, ["name", "age"]);
// Type: (string | number)[]

// Constrained generic with specific key type
function updateProperty<T extends object, K extends keyof T>(
  obj: T,
  key: K,
  value: T[K]
): T {
  return { ...obj, [key]: value };
}

const updated = updateProperty(user, "name", "Jane");
// TypeScript ensures value matches the property type
// updateProperty(user, "age", "thirty"); // ❌ Error: string not assignable to number
```

### 22.2.2 Generic Object Manipulation

Create flexible utilities that work with any object shape using `keyof` and generics.

```typescript
// Type-safe pick function (like Pick but runtime)
function pick<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): Pick<T, K> {
  const result = {} as Pick<T, K>;
  
  keys.forEach(key => {
    if (key in obj) {
      result[key] = obj[key];
    }
  });
  
  return result;
}

const user = {
  id: 1,
  name: "John",
  email: "john@example.com",
  age: 30
};

const userPreview = pick(user, ["id", "name"]);
// Type: { id: number; name: string; }
// userPreview.email would be a compile-time error

// Type-safe omit function
function omit<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): Omit<T, K> {
  const result = { ...obj };
  keys.forEach(key => delete (result as any)[key]);
  return result;
}

const publicUser = omit(user, ["email", "age"]);
// Type: { id: number; name: string; }

// Transform object values
function mapValues<T extends object, V>(
  obj: T,
  fn: (value: T[keyof T], key: keyof T) => V
): Record<keyof T, V> {
  const result = {} as Record<keyof T, V>;
  
  (Object.keys(obj) as Array<keyof T>).forEach(key => {
    result[key] = fn(obj[key], key);
  });
  
  return result;
}

const stringified = mapValues(user, (value) => String(value));
// Type: { id: string; name: string; email: string; age: string; }
```

### 22.2.3 Advanced Generic Patterns

Combine `keyof` with mapped types and conditional types for sophisticated transformations.

```typescript
// Make specific keys optional
type PartialBy<T, K extends keyof T> = Partial<Pick<T, K>> & Omit<T, K>;

interface User {
  id: number;
  name: string;
  email: string;
  age: number;
}

type UserWithOptionalEmail = PartialBy<User, "email" | "age">;
// { email?: string; age?: number; } & { id: number; name: string; }

// Function to implement this at runtime
function makePartial<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): PartialBy<T, K> {
  return { ...obj }; // Implementation would handle optional logic
}

// Extract keys by value type
type KeysOfType<T, V> = {
  [K in keyof T]: T[K] extends V ? K : never;
}[keyof T];

interface Mixed {
  name: string;
  age: number;
  email: string;
  active: boolean;
}

type StringKeys = KeysOfType<Mixed, string>;
// "name" | "email"

// Pick by value type
type PickByType<T, V> = Pick<T, KeysOfType<T, V>>;

type StringProps = PickByType<Mixed, string>;
// { name: string; email: string; }

// Ensure all keys of T exist in U (type compatibility check)
type HasAllKeys<T, U> = keyof T extends keyof U ? true : false;

interface A { a: string; b: number; }
interface B { a: string; b: number; c: boolean; }
interface C { a: string; }

type Check1 = HasAllKeys<A, B>; // true (A's keys are in B)
type Check2 = HasAllKeys<B, A>; // false (B has 'c' which A lacks)
type Check3 = HasAllKeys<C, A>; // true (C's key 'a' is in A)
```

---

## 22.3 `keyof` with Union Types

When applied to union types, `keyof` behaves differently than you might expect, returning only the common keys shared by all members of the union.

### 22.3.1 Union Key Intersection

`keyof` applied to a union type returns the intersection of keys (keys present in ALL union members), not the union of all keys.

```typescript
// Union of object types
type Animal = {
  name: string;
  species: string;
};

type Plant = {
  name: string;
  genus: string;
};

type Organism = Animal | Plant;

type OrganismKeys = keyof Organism;
// "name" only!
// Not "name" | "species" | "genus"

// Why? Because TypeScript doesn't know which one you'll have at runtime
function getOrganismName(org: Organism): string {
  return org.name; // ✅ Safe: both have name
  // return org.species; // ❌ Error: Plant doesn't have species
}

// To get all possible keys, use distributive conditional
type AllKeys<T> = T extends any ? keyof T : never;
type AllOrganismKeys = AllKeys<Organism>;
// "name" | "species" | "genus"

// Accessing union type properties safely
function printOrganismInfo(org: Organism) {
  console.log(org.name); // Always safe
  
  // Type guard required for specific properties
  if ("species" in org) {
    console.log(org.species); // TypeScript knows this is Animal
  } else {
    console.log(org.genus); // TypeScript knows this is Plant
  }
}
```

### 22.3.2 `keyof` with Intersection Types

With intersection types, `keyof` returns the union of all keys from all intersected types.

```typescript
// Intersection type
type Person = {
  name: string;
  age: number;
};

type Employee = {
  employeeId: string;
  department: string;
};

type EmployeePerson = Person & Employee;

type EmployeePersonKeys = keyof EmployeePerson;
// "name" | "age" | "employeeId" | "department"

// Practical use: Merging configurations
type DefaultConfig = {
  timeout: number;
  retries: number;
};

type UserConfig = {
  apiUrl: string;
  apiKey: string;
};

type Config = DefaultConfig & UserConfig;
type ConfigKey = keyof Config;
// "timeout" | "retries" | "apiUrl" | "apiKey"

function setupConfig(config: Partial<Config>): Config {
  return {
    timeout: 5000,
    retries: 3,
    apiUrl: "https://api.example.com",
    apiKey: "",
    ...config
  };
}
```

### 22.3.3 Handling Optional Properties

`keyof` includes optional properties in the union, but accessing them requires consideration of undefined values.

```typescript
interface OptionalUser {
  id: number;
  name: string;
  email?: string;
  phone?: string;
}

type OptionalUserKeys = keyof OptionalUser;
// "id" | "name" | "email" | "phone"

// Accessing optional properties
function getUserInfo(user: OptionalUser, key: keyof OptionalUser) {
  const value = user[key];
  // Type: string | number | undefined
  
  if (key === "id" || key === "name") {
    // TypeScript doesn't narrow here automatically
    // value is still string | number | undefined
  }
  
  return value;
}

// Better approach: distinguish required vs optional
type RequiredKeys<T> = {
  [K in keyof T]-?: {} extends Pick<T, K> ? never : K;
}[keyof T];

type OptionalKeys<T> = {
  [K in keyof T]-?: {} extends Pick<T, K> ? K : never;
}[keyof T];

type Req = RequiredKeys<OptionalUser>; // "id" | "name"
type Opt = OptionalKeys<OptionalUser>; // "email" | "phone"
```

---

## 22.4 Dynamic Property Access

`keyof` enables type-safe dynamic property access patterns that would otherwise require unsafe type assertions or any types.

### 22.4.1 Index Access Types

Combine `keyof` with bracket notation to look up specific property types dynamically.

```typescript
interface User {
  id: number;
  name: string;
  email: string;
  metadata: {
    createdAt: Date;
    updatedAt: Date;
  };
}

// Lookup specific property type
type UserId = User["id"];           // number
type UserMetadata = User["metadata"]; // { createdAt: Date; updatedAt: Date; }

// Lookup with union of keys
type UserFields = User["id" | "name"]; // number | string

// Lookup nested property
type CreatedAtType = User["metadata"]["createdAt"]; // Date

// Dynamic lookup with generic
function getNestedValue<T extends object, K extends keyof T>(
  obj: T,
  key: K
): T[K] {
  return obj[key];
}

// Accessing array elements
interface Database {
  users: User[];
  settings: Record<string, string>;
}

type UsersArray = Database["users"]; // User[]
type FirstUser = Database["users"][number]; // User
type SettingValue = Database["settings"][string]; // string
```

### 22.4.2 Type-Safe Property Paths

Create utilities for accessing nested properties safely using recursive `keyof` patterns.

```typescript
// Simple path type (one level)
type Path<T> = keyof T;

// Two-level path
type Path2<T> = {
  [K in keyof T]: K extends string
    ? T[K] extends object
      ? `${K}` | `${K}.${keyof T[K] extends string ? keyof T[K] : never}`
      : `${K}`
    : never;
}[keyof T];

interface Config {
  server: {
    host: string;
    port: number;
  };
  database: {
    url: string;
    pool: {
      min: number;
      max: number;
    };
  };
}

type ConfigPaths = Path2<Config>;
// "server" | "server.host" | "server.port" | "database" | "database.url" | "database.pool"

// Deep path extraction (simplified)
type DeepPath<T, K extends keyof T = keyof T> = K extends string
  ? T[K] extends object
    ? `${K}` | `${K}.${DeepPath<T[K]>}`
    : `${K}`
  : never;

// Type-safe getter for paths
function getPath<T extends object, P extends DeepPath<T>>(
  obj: T,
  path: P
): any {
  return path.split('.').reduce((acc, part) => acc?.[part], obj);
}

const config: Config = {
  server: { host: "localhost", port: 3000 },
  database: { url: "postgres://localhost", pool: { min: 2, max: 10 } }
};

const host = getPath(config, "server.host"); // TypeScript knows this should exist
const pool = getPath(config, "database.pool"); // Returns { min: number; max: number; }
```

### 22.4.3 Key Remapping

Use `keyof` with mapped types to remap keys while preserving type relationships.

```typescript
// Prefix all keys
type AddPrefix<T, P extends string> = {
  [K in keyof T as `${P}${string & K}`]: T[K];
};

interface User {
  name: string;
  age: number;
}

type ApiUser = AddPrefix<User, "user_">;
// { user_name: string; user_age: number; }

// Transform keys to camelCase from snake_case (simplified)
type CamelCase<S extends string> = S extends `${infer P}_${infer C}${infer R}`
  ? `${P}${Uppercase<C>}${CamelCase<R>}`
  : S;

type CamelKeys<T> = {
  [K in keyof T as CamelCase<string & K>]: T[K];
};

interface SnakeUser {
  first_name: string;
  last_name: string;
}

type CamelUser = CamelKeys<SnakeUser>;
// { firstName: string; lastName: string; }

// Filter keys by value type
type FilterKeys<T, V> = {
  [K in keyof T as T[K] extends V ? K : never]: T[K];
};

interface Mixed {
  id: number;
  name: string;
  count: number;
  active: boolean;
}

type NumberKeys = FilterKeys<Mixed, number>;
// { id: number; count: number; }
```

---

## 22.5 Type-Safe Object Keys

`keyof` enables patterns for ensuring object keys are handled safely throughout your codebase, from function parameters to API contracts.

### 22.5.1 Enforcing Key Existence

Ensure that functions only accept keys that actually exist on objects, preventing runtime errors.

```typescript
// Strict key checking
function pluck<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): Pick<T, K> {
  const result = {} as Pick<T, K>;
  
  for (const key of keys) {
    if (key in obj) {
      result[key] = obj[key];
    }
  }
  
  return result;
}

// Usage
const user = {
  id: 1,
  name: "John",
  email: "john@example.com"
};

// TypeScript validates keys at compile time
const subset = pluck(user, ["id", "name"]); // ✅ OK
// const bad = pluck(user, ["id", "phone"]); // ❌ Error: "phone" doesn't exist

// Key existence check type
type HasKey<T, K extends PropertyKey> = K extends keyof T ? true : false;

type Check1 = HasKey<User, "name">;  // true
type Check2 = HasKey<User, "phone">; // false

// Enforcing complete key sets
function requireKeys<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): asserts obj is Required<Pick<T, K>> & T {
  const missing = keys.filter(key => !(key in obj));
  if (missing.length > 0) {
    throw new Error(`Missing keys: ${missing.join(", ")}`);
  }
}

// Usage
const partialUser: Partial<User> = { id: 1, name: "John" };
requireKeys(partialUser, ["id", "name", "email"]);
// Now TypeScript knows partialUser has all three properties
```

### 22.5.2 Key-Value Relationships

Maintain relationships between keys and values in type-safe mappings.

```typescript
// Type-safe event handlers
type EventMap = {
  click: { x: number; y: number };
  submit: { data: FormData };
  cancel: { reason: string };
};

type EventNames = keyof EventMap;

function addEventListener<K extends EventNames>(
  event: K,
  handler: (payload: EventMap[K]) => void
): void {
  // Implementation
}

// TypeScript knows the payload type based on event name
addEventListener("click", (payload) => {
  console.log(payload.x, payload.y); // ✅ Typed as { x: number; y: number }
});

addEventListener("submit", (payload) => {
  console.log(payload.data); // ✅ Typed as { data: FormData }
});

// Type-safe configuration
type ConfigSchema = {
  timeout: number;
  retries: number;
  endpoint: string;
};

type ConfigValues = ConfigSchema[keyof ConfigSchema]; // number | string

function setConfig<K extends keyof ConfigSchema>(
  key: K,
  value: ConfigSchema[K]
): void {
  // Implementation ensures type safety
}

setConfig("timeout", 5000);     // ✅ OK
setConfig("retries", 3);        // ✅ OK
setConfig("endpoint", "/api");  // ✅ OK
// setConfig("timeout", "5000"); // ❌ Error: string not assignable to number
```

### 22.5.3 Dictionary and Map Types

Create type-safe dictionaries where keys are constrained to specific sets.

```typescript
// Strict dictionary
type StrictDictionary<K extends string, V> = {
  [P in K]: V;
};

type UserRoles = "admin" | "editor" | "viewer";
type Permissions = StrictDictionary<UserRoles, string[]>;

const permissions: Permissions = {
  admin: ["read", "write", "delete"],
  editor: ["read", "write"],
  viewer: ["read"],
  // guest: ["read"] // ❌ Error: "guest" not in UserRoles
};

// Partial dictionary (optional keys)
type PartialDictionary<K extends string, V> = {
  [P in K]?: V;
};

// Record with constrained keys (alternative to StrictDictionary)
type RolePermissions = Record<UserRoles, string[]>;

// Reverse mapping
type ReverseMapping<T extends Record<string, string>> = {
  [K in keyof T as T[K]]: K;
};

type RoleCodes = {
  admin: "A";
  editor: "E";
  viewer: "V";
};

type CodeToRole = ReverseMapping<RoleCodes>;
// { A: "admin"; E: "editor"; V: "viewer"; }
```

---

## 22.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored the `keyof` operator, a fundamental tool for type-level programming in TypeScript:

**Key Takeaways:**

1. **Basic `keyof` Usage**:
   - Returns a union of all public property keys of a type
   - Works with interfaces, type aliases, classes, and objects
   - Returns `string | number | symbol` for index signatures

2. **Generics with `keyof`**:
   - Constrain generic parameters with `K extends keyof T`
   - Enable type-safe property access in generic functions
   - Combine with mapped types for powerful transformations

3. **Union and Intersection Behavior**:
   - `keyof (A | B)` returns intersection of keys (common keys only)
   - `keyof (A & B)` returns union of all keys
   - Use distributive conditionals to get all keys from unions

4. **Dynamic Property Access**:
   - Index access types: `T[K]` where `K extends keyof T`
   - Build type-safe paths for nested objects
   - Remap keys using template literal types with `keyof`

5. **Type-Safe Patterns**:
   - Enforce key existence at compile time
   - Maintain key-value relationships in mappings
   - Create strict dictionaries with constrained keys

### Practical Exercises

**Exercise 1: Basic `keyof` Operations**

Practice extracting and using keys:

```typescript
interface Product {
  id: number;
  name: string;
  price: number;
  category: string;
  inStock: boolean;
}

// 1. Extract all keys of Product into a union type
type ProductKeys = /* your code */;

// 2. Create a type that is the union of all value types in Product
type ProductValues = /* your code */;

// 3. Create a function that accepts only valid Product keys
function logProperty(product: Product, key: /* your code */): void {
  console.log(product[key]);
}

// 4. Create a type with only the number-valued keys from Product
type NumericProductProps = /* your code */;
```

**Exercise 2: Generic Functions with `keyof`**

Build type-safe generic utilities:

```typescript
// 1. Implement a type-safe pick function
function pick<T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): Pick<T, K> {
  // Implementation
}

// 2. Implement a type-safe get function that handles nested paths
function get<T extends object, P extends string>(
  obj: T,
  path: P
): /* return type based on path */ {
  // Implementation should handle dot notation like "user.name"
}

// 3. Create a function that swaps keys and values (for string-valued objects)
function invert<T extends Record<string, string>>(
  obj: T
): /* return type */ {
  // Implementation
}

// 4. Create a type-safe event emitter where event names are constrained
// to keys of an event map, and payloads are typed accordingly
class TypedEmitter<Events extends Record<string, any>> {
  emit<K extends keyof Events>(event: K, payload: Events[K]): void {
    // Implementation
  }
}
```

**Exercise 3: Advanced `keyof` Patterns**

Implement advanced type manipulations:

```typescript
// 1. Create a DeepPartial type that makes all properties optional recursively
type DeepPartial<T> = /* your code */;

// 2. Create a type that extracts only the required keys from T
type RequiredKeys<T> = /* your code */;

// 3. Create a type that makes specific keys required while keeping others optional
type RequiredBy<T, K extends keyof T> = /* your code */;

// 4. Implement a type-safe rename function that renames keys while preserving types
function renameKeys<T extends object, M extends Record<keyof T, string>>(
  obj: T,
  mapping: M
): /* return type with renamed keys */ {
  // Implementation
}

// Test case:
// renameKeys({ a: 1, b: "hello" }, { a: "x", b: "y" })
// should return type { x: number; y: string; }
```

**Exercise 4: Real-World Application**

Build a type-safe database query builder:

```typescript
// Define a schema
interface UserSchema {
  id: number;
  name: string;
  email: string;
  age: number;
  createdAt: Date;
}

// Implement a QueryBuilder that:
// 1. Only allows selecting fields that exist in the schema
// 2. Type-checks where clauses against field types
// 3. Returns correctly typed results

class QueryBuilder<T extends object> {
  private table: string;
  
  constructor(table: string) {
    this.table = table;
  }
  
  select<K extends keyof T>(...fields: K[]): QueryBuilder<Pick<T, K>> {
    // Implementation
    return this;
  }
  
  where<K extends keyof T>(
    field: K,
    operator: "=" | ">" | "<" | ">=" | "<=",
    value: T[K]
  ): this {
    // Implementation
    return this;
  }
  
  execute(): Promise<T[]> {
    // Implementation
    return Promise.resolve([]);
  }
}

// Usage should be fully type-safe:
const users = await new QueryBuilder<UserSchema>("users")
  .select("id", "name", "email")
  .where("age", ">", 18)
  .where("name", "=", "John")
  .execute();

// users should be typed as Array<{ id: number; name: string; email: string; }>
```

**Exercise 5: Key Remapping and Transformation**

Create utilities for transforming object keys:

```typescript
// 1. Create a type that prefixes all keys with a string
type PrefixKeys<T, P extends string> = /* your code */;

// 2. Create a type that converts all keys to PascalCase
type PascalCase<S extends string> = /* your code */;
type PascalKeys<T> = /* your code */;

// 3. Create a type that groups keys by their value type
type GroupByType<T> = /* your code */;
// Should produce something like:
// { string: { name: string; email: string; }; number: { age: number; }; }

// 4. Implement a diff type that shows what keys differ between two objects
type KeyDiff<T, U> = /* your code */;
// Should return keys that exist in T but not U, and vice versa
```

### Additional Resources

- **TypeScript Handbook - Index Types**: https://www.typescriptlang.org/docs/handbook/2/indexed-access-types.html
- **TypeScript Handbook - Mapped Types**: https://www.typescriptlang.org/docs/handbook/2/mapped-types.html
- **Type Challenges**: https://github.com/type-challenges/type-challenges (Look for keyof-related challenges)
- **Total TypeScript**: https://www.totaltypescript.com/concepts/keyof

---

## Coming Up Next: Chapter 23 - Type Inference

In the next chapter, we will explore **Type Inference**, the mechanism that allows TypeScript to automatically deduce types without explicit annotations:

- **23.1 How Type Inference Works** - The algorithms behind TypeScript's type deduction
- **23.2 Best Common Type Algorithm** - How TypeScript combines multiple types
- **23.3 Return Type Inference** - How function return types are inferred
- **23.4 Contextual Typing** - How context influences type inference
- **23.5 Type Inference in Arrays** - Array literal type deduction
- **23.6 Inferring in Conditional Types** - Advanced inference patterns
- **23.7 Controlling Inference** - Techniques to guide TypeScript's inference

Understanding type inference is crucial for writing concise TypeScript code without sacrificing type safety. It explains why TypeScript often "just knows" the right types without explicit annotations, and how to leverage this power effectively in your code.